In [1]:
import os
import pandas as pd
import requests
from sklearn.metrics import mean_absolute_error
import numpy as np

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


Num GPUs Available:  0


IndexError: list index out of range

In [3]:
# 10-15 countries to start training
# Check whether the data file is updated and if we can train in predict.py

# Goal
Predict the daily number of cases for the given countries, regions and intervention plan.

This model is described in [From Prediction to Prescription: Evolutionary Optimization of Non-Pharmaceutical Interventions in the COVID-19 Pandemic](https://arxiv.org/abs/2005.13766) in Section 5 - Data-Driven Predictive Model.

Predictions from this model trained on the latest data are visible here: [How AI Makes Intervention Recommendations](https://evolution.ml/demos/npidashboard/). For more information see [Augmenting Human Decision Making
Optimizing COVID-19 Interventions](https://evolution.ml/esp/npi/).

## Intervention plan
An intervention plan consists in a list of containment and closure policies, as well as health system policies. See https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md for more details. The following policies are considered:

In [4]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']

## Example
For the given input, provide a model that can produce the expected output

### Given input
One row represents a country, region and day. For each row, the interventions that were in place on this day for this region and country are provided. Interventions can change from one day to the other for each country and region.

In [5]:
EXAMPLE_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"

In [6]:
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

In [7]:
prediction_input_df.head()

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Expected output
For each row that was provided in input, i.e. for each country, region and day, the output should contain an additional `PredictedDailyNewCases` column with the predicted number of cases for that day, region and country. It is possible to leave `PredictedDailyNewCases` empty or NaN, or to remove the row, in case no predition is available.

In [8]:
EXAMPLE_OUTPUT_FILE = "../../../../2020-08-01_2020-08-04_predictions_example.csv"

In [9]:
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")

In [10]:
prediction_output_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
0,Aruba,NaN,2020-08-01,0.820071
1,Aruba,NaN,2020-08-02,0.872854
2,Aruba,NaN,2020-08-03,0.000000
3,Aruba,NaN,2020-08-04,0.000000
4,Afghanistan,NaN,2020-08-01,80.590128


### Evaluation
Predictions will be evaluated on a period of 4 weeks **after** submision against the actual daily change in confirmed cases reported by the [Oxford COVID-19 Government Response Tracker (OxCGRT)](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker).

The latest data, including the latest confirmed cases ('ConfirmedCases') can be find here: https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv

In [11]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [12]:
df.sample(3)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
33505,Estonia,EST,NaN,NaN,NAT_TOTAL,2020-11-19,1.0,1.0,1.0,1.0,...,37.96,37.96,53.57,53.57,40.56,40.56,39.10,39.10,50.0,50.0
49621,Libya,LBY,NaN,NaN,NAT_TOTAL,2020-07-20,3.0,1.0,2.0,1.0,...,90.74,90.74,96.43,96.43,60.00,60.00,69.23,69.23,0.0,0.0
73455,Timor-Leste,TLS,NaN,NaN,NAT_TOTAL,2020-02-01,0.0,NaN,0.0,NaN,...,5.56,5.56,9.52,9.52,3.33,3.33,3.85,3.85,0.0,0.0


### Daily change in confirmed cases
The daily change in confirmed cases can be computed like this:

In [13]:
df["GeoID"] = np.where(df["RegionName"].isnull(),
                                      df["CountryName"],
                                      df["CountryName"] + ' / ' + df["RegionName"])
df["DailyChangeConfirmedCases"] = df.groupby(["GeoID"]).ConfirmedCases.diff().fillna(0)

For instance, for country **United States**, region **California**, the latest available changes in confirmed cases are:

In [14]:
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]

In [15]:
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
79067,United States,California,2020-12-14,1611493.0,26115.0
79068,United States,California,2020-12-15,1644742.0,33249.0
79069,United States,California,2020-12-16,1708559.0,63817.0
79070,United States,California,2020-12-17,1753974.0,45415.0
79071,United States,California,2020-12-18,NaN,0.0


# Training a model

## Copy the data locally

In [16]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_FILE = 'data/OxCGRT_latest.csv'

# Download the data set
data = requests.get(DATA_URL)

# Persist the data set locally in order to use it after submission to make predictions,
# as the sandbox won't have access to the internet anymore.
if not os.path.exists('data'):
    os.mkdir('data')
open(DATA_FILE, 'wb').write(data.content)

20719256

## Train

In [23]:
# Reload the module to get the latest changes
# import xprize_predictor
# from importlib import reload
# reload(xprize_predictor)
# from xprize_predictor import XPrizePredictor
import xprize_predictor_copy
from importlib import reload
reload(xprize_predictor_copy)
from xprize_predictor_copy import XPrizePredictor

In [24]:
predictor = XPrizePredictor(None, DATA_FILE)

Loading holiday data


In [25]:
%%time
predictor_model = predictor.train()

Creating numpy arrays for Keras for each country...
Numpy arrays created
Trial 0
14/14 [==============================] - 0s 2ms/step - loss: 0.0312
Train Loss: 0.05629141628742218
Val Loss: 0.055013272911310196
Test Loss: 0.031224068254232407
Done
CPU times: user 18min 22s, sys: 3min 37s, total: 21min 59s
Wall time: 4min 37s


In [26]:
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights("models/holidays_confirmed_deaths.h5")

# Predicting using a trained model

## Load candidate model

In [21]:
model_weights_file = "models/trained_model_weights.h5"

In [22]:
predictor = XPrizePredictor(model_weights_file, DATA_FILE)

## Make prediction

In [61]:
NPIS_INPUT_FILE = "../../../validation/data/2020-12-16_historical_ip.csv"
start_date = "2020-12-01"
end_date = "2020-12-15"

In [24]:
%%time
preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 1min 59s, sys: 2.69 s, total: 2min 1s
Wall time: 1min 53s


In [85]:
# preds_date_df = preds_df[(preds_df['Date'] >= '2020-08-01') & (preds_df['Date'] <= '2020-08-31')]
date_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
california_df = date_df[(date_df.CountryName == "United States") & (date_df.RegionName == "California")]

In [26]:
merged_df = pd.merge(date_df, preds_date_df, on=["Date", "CountryName", "RegionName"])
mse = mean_squared_error(merged_df['DailyChangeConfirmedCases'], merged_df['PredictedDailyNewCases'])
# merged_df['NewCasesDifference'] = merged_df['PredictedDailyNewCases'] - merged_df['DailyChangeConfirmedCases']
# merged_df[["CountryName", "RegionName", "Date", "DailyChangeConfirmedCases", "PredictedDailyNewCases", "NewCasesDifference"]].head()

Change test days to 60

In [27]:
model_60_weights_file = "models/trained_model_weights_60_test_days.h5"

In [28]:
predictor = XPrizePredictor(model_60_weights_file, DATA_FILE)

In [76]:
model_weights_original = "models/original_model.h5"
predictor = XPrizePredictor(model_weights_original, DATA_FILE)

      CountryName CountryCode RegionName RegionCode Jurisdiction       Date  \
98335    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-07   
98336    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-08   
98337    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-09   
98338    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-10   
98339    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-11   
98340    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-12   
98341    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-13   
98342    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-14   
98343    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-15   
98344    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-16   
98345    Zimbabwe         ZWE        NaN        NaN    NAT_TOTAL 2020-05-17   
98346    Zimbabwe         ZWE        NaN        NaN 

In [77]:
%%time
preds_original_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

      CountryName RegionName       Date  C1_School closing  \
93142    Zimbabwe        NaN 2020-05-07                3.0   
93143    Zimbabwe        NaN 2020-05-08                3.0   
93144    Zimbabwe        NaN 2020-05-09                3.0   
93145    Zimbabwe        NaN 2020-05-10                3.0   
93146    Zimbabwe        NaN 2020-05-11                3.0   
93147    Zimbabwe        NaN 2020-05-12                3.0   
93148    Zimbabwe        NaN 2020-05-13                3.0   
93149    Zimbabwe        NaN 2020-05-14                3.0   
93150    Zimbabwe        NaN 2020-05-15                3.0   
93151    Zimbabwe        NaN 2020-05-16                3.0   
93152    Zimbabwe        NaN 2020-05-17                3.0   
93153    Zimbabwe        NaN 2020-05-18                3.0   
93154    Zimbabwe        NaN 2020-05-19                3.0   
93155    Zimbabwe        NaN 2020-05-20                3.0   

       C2_Workplace closing  C3_Cancel public events  \
93142        

In [101]:
#BASELINE
preds_original_date_df = preds_original_df[(preds_original_df["Date"] >= start_date) & (preds_original_df["Date"] <= end_date)]
full_df = df[(df["Date"] <= end_date)]
preds_original_date_df["GeoID"] = np.where(preds_original_date_df["RegionName"].isnull(),
                                      preds_original_date_df["CountryName"],
                                      preds_original_date_df["CountryName"] + ' / ' + preds_original_date_df["RegionName"])
merged_original_df = pd.merge(date_df, preds_original_date_df, on=["Date", "GeoID"])
gdf = full_df.groupby('GeoID')['ConfirmedDeaths'].agg(['max', 'count']).sort_values(by='max', ascending=False)
filtered_gdf = gdf[gdf["count"] > 21]
geos = list(filtered_gdf.head(20).index)
merged_original_df = merged_original_df[(merged_original_df["GeoID"].isin(geos))]
preds_original_mae = np.sum(np.abs(merged_original_df.DailyChangeConfirmedCases.values - merged_original_df.PredictedDailyNewCases.values))
preds_original_mae

285692.6288653585

In [110]:
model_weights_pred_ratio = "models/original_model_prediction_ratio.h5"
predictor = XPrizePredictor(model_weights_pred_ratio, DATA_FILE)

In [111]:
%%time
preds_ratio_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 1min 2s, sys: 1.28 s, total: 1min 3s
Wall time: 59.1 s


In [112]:
#PREDICTION RATIO
preds_ratio_date_df = preds_ratio_df[(preds_ratio_df["Date"] >= start_date) & (preds_ratio_df["Date"] <= end_date)]
preds_ratio_date_df["GeoID"] = np.where(preds_ratio_date_df["RegionName"].isnull(),
                                      preds_ratio_date_df["CountryName"],
                                      preds_ratio_date_df["CountryName"] + ' / ' + preds_ratio_date_df["RegionName"])
merged_ratio_df = pd.merge(date_df, preds_ratio_date_df, on=["Date", "GeoID"])
merged_ratio_df = merged_ratio_df[(merged_ratio_df["GeoID"].isin(geos))]
preds_ratio_mae = np.sum(np.abs(merged_ratio_df.DailyChangeConfirmedCases.values - merged_ratio_df.PredictedDailyNewCases.values))
preds_ratio_mae

189715.16666472307

In [73]:
# BETTER PREDICT FUNCTION
model_weights_pred_ratio = "models/original_model_prediction_ratio.h5"
better_predictor = XPrizePredictor(model_weights_pred_ratio, DATA_FILE)


In [74]:
%%time
better_preds_ratio_df = better_predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 15min 19s, sys: 3.01 s, total: 15min 22s
Wall time: 15min 19s


In [75]:
#PREDICTION RATIO BETTER PREDICTOR
better_preds_ratio_date_df = better_preds_ratio_df[(better_preds_ratio_df["Date"] >= start_date) & (better_preds_ratio_df["Date"] <= end_date)]
better_preds_ratio_date_df["GeoID"] = np.where(better_preds_ratio_date_df["RegionName"].isnull(),
                                      better_preds_ratio_date_df["CountryName"],
                                      better_preds_ratio_date_df["CountryName"] + ' / ' + better_preds_ratio_date_df["RegionName"])
better_merged_ratio_df = pd.merge(date_df, better_preds_ratio_date_df, on=["Date", "GeoID"])
better_merged_ratio_df = better_merged_ratio_df[(better_merged_ratio_df["GeoID"].isin(geos))]
better_preds_ratio_mae = np.sum(np.abs(better_merged_ratio_df.DailyChangeConfirmedCases.values - better_merged_ratio_df.PredictedDailyNewCases.values))
better_preds_ratio_mae

NameError: name 'date_df' is not defined

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [21]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [22]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,0.0
Aruba,,2020-08-02,1.7698189576811538
Aruba,,2020-08-03,21.939474736193304
Aruba,,2020-08-04,25.106195049205034
Afghanistan,,2020-08-01,269.09904415313025
Afghanistan,,2020-08-02,155.55027796759626
Afghanistan,,2020-08-03,240.02544615184394
Afghanistan,,2020-08-04,200.2726904410946
Angola,,2020-08-01,15.565491775948328


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [24]:
# Check the pediction file is valid
import os
from predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [39]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [40]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 11.6 s, sys: 1.99 s, total: 13.6 s
Wall time: 18min 25s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [41]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-13
End date: 2021-06-11


In [42]:
from scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [43]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-13 to 2021-06-11...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 19.8 s, sys: 3.12 s, total: 22.9 s
Wall time: 29min 47s
